## Stock span

We are given list price of a stock for N days,  find the stock span of each day. Stock span is defined as the consecutive number of prior days where the stock price was less or equal to the given day stock price.

For ex. for   P  = {11, 9, 7, 5, 4, 6, 8, 10, 7, 9}  the span is S = {1, 1, 1, 1, 1, 3, 4, 7, 1, 2}

![](img/stockSpan.jpeg)

### Algorithm:
1. Inititalize span\[0\]=1 and stack.push(0) (Index of first day price)

2. Iterate from 2nd day to end and pop all entries from stack that are less than current day

    a. Set span\[i\]=stack.peek()-i or i if stack is empty. Pushing the index into the stack enables you to do this since you need to count days for span

    b. push i into stack 

The intuition behind the former approach is that for each iteration you only have prior greater day price's index in stack. As soon as you come to a new maximum you pop all the previous smaller prices off the stack

https://www.youtube.com/watch?v=-IFmgue8sF0

In [1]:
def spanNaive(arr):
    span=[1]*len(arr)
    for i in range(len(arr)):
        j=i-1
        while j>=0 and arr[j] <= arr[i]:
            span[i] += 1
            j-=1
    return spans

def span(arr):
    span=[None]*len(arr)
    span[0]=1 #Least span is always 1
    stack=[]
    stack.append(0)
    for i in range(1,len(arr)):
        while stack and arr[stack[-1]] < arr[i]: #First pop everything thats smaller than i
            stack.pop()
        span[i] = i - stack[-1] if stack else i+1 #If stack is empty then span is entire array till i, Ternary operations in Python
        #If non empty then above formula is intuitive
        #Note that you are pushing index of i into stack and not the element itself. Because u need to use the index
        stack.append(i)
    return span

print(span([11, 9, 7, 5, 4, 6, 8, 10, 7, 9]))

[1, 1, 1, 1, 1, 3, 5, 7, 1, 2]


## Largest Area in a Histogram

 Given n non-negative integers representing the histogram's bar height where the width of each bar is 1, find the area of largest rectangle in the histogram.

In the above histogram width of each bar is 1 and heights are H =\[2, 1, 5, 6, 2, 3\]. 


Solution: 

The intuition in this problem is to calculate the larges area computed by including that rectangle fully. And then return the largest of all these rectangles

Algorithm :

1. Create a empty stack.   

2. Push i , if stack is empty or H\[i\] is greater than stack.peek(). 
3. If the bar is small set anchor to i, continue to pop the indces from the stack until H\[stack.peek()\] is smaller H\[i\], for each element poped calculate the area evert time you pop element using  

A = bar_height(of the poped index) * (i - 1 - stack.peek())   

when stack is empty,   
A = bar_height(of the poped index) * i. 

update Area A if the newly computed Area is greater than previous.   

4. If all the items are over and stack is still not empty repeat process 3.  


![](img/maxHistogram.png)

In [2]:
def maxHistogram(arr):
    maxArea=float('-inf')
    area=0
    stack=[]
    i=0
    #This algo illustrates a new technique to conditionally iterate through a list
    while i < len(arr):
        #Doing a not/or combination below removes the necessity for a check in the else section
        if not stack or arr[stack[-1]] < arr[i]: #Remember no while here, only push each element that is greater than top for each iteration
            stack.append(i)
            i+=1 #Note that i only gets incremented here
        else:
            top = stack.pop()
            if stack:
                area = arr[top]*(i-stack[-1]-1) #This formula is tricky (i - peek -1)
            else:
                area = arr[top]*i
            maxArea= max(maxArea, area) #Note maxArea gets computed only here and not indented left scope

    while stack:
        top = stack.pop()
        area = arr[top]*(i-stack[-1]-1) if stack else arr[top]*i
        maxArea= max(maxArea, area)

    return maxArea

print(maxHistogram([2, 1, 5, 6, 2, 3]))

10


## Maximal Rectangle

Given a rows x cols binary matrix filled with 0's and 1's, find the largest rectangle containing only 1's and return its area.

Input: matrix = \[\["1","0","1","0","0"\],\["1","0","1","1","1"\],\["1","1","1","1","1"\],\["1","0","0","1","0"\]\]
Output: 6
Explanation: The maximal rectangle is shown in the above picture.

Solution: Compute the height sum for each row where the cell is not zero and then call maxHistogram for each row and return the maxArea


In [3]:
def maxRectangle(rect):
    area=0
    maxArea=float('-inf')
    for i in range(len(rect)):
        row = [0]*len(rect[i])
        for j in range(len(rect[i])):
            if rect[i][j]==0:
                row[j]=0
            else:
                for k in range(i+1):
                    row[j] += rect[k][j]
        area=maxHistogram(row)
        maxArea = max(maxArea, area)
    return maxArea

rect=[[1,0,1,0,0],
      [1,0,1,1,1],
      [1,1,1,1,1],
      [1,0,0,1,0]]

print(maxRectangle(rect))

6


## Trapping rain water

Given n non-negative integers representing an elevation map where the width of each bar is 1, compute how much water it can trap after raining.

Hard, Variation of 2 pointer where pointers need to be at 2 ends. Approach below with both pointers starting at same end is not solvable

In [4]:
def trappedWaterIncomplete(num):
    if len(num) == 0:
        return 0
    i=j=0
    waterSoFar=totalWater=0
    while j<len(num):
        if num[j] >= num[i]:
            totalWater += waterSoFar
            waterSoFar=0
            i=j
        if num[j] < num[i]:
            waterSoFar += num[i]-num[j]
        j+=1
    return totalWater

print("Total Water = " + str(trappedWaterIncomplete([0, 1, 0, 4, 1, 0, 1, 3, 2, 1, 2, 1])))

Total Water = 1


In [8]:
"""
Simpler approach with extra space - 
Approach: we just need to focus on water over a particular bar.
This boils down to the problem of knowing local maxima on both sides of the bar.
Precompute this local maxima in two extra arrays
"""
#using prefix and postfix arrays for local maximas
def getTrappedWater(heights):
    if not heights: return 0
    left, right = [],[]
    maxx = float('-inf')
    for height in heights:
        maxx = max(maxx,height)
        left.append(maxx)

    #re-initialize maxx variable for traversing list in reverse dxn
    maxx = float('-inf')
    for i in range(len(heights)-1,-1,-1):
        maxx = max(maxx,heights[i])
        right.append(maxx)

    #reversing for makng the order correct.
    right.reverse()

    #boundary buildings have no water over it, so we can skip those in loop for calculating ans
    result = 0
    for i in range(1,len(heights)-1):
        result+=(min(left[i],right[i])-heights[i])
    return result

print(getTrappedWater([0, 1, 0, 2, 1, 0, 1, 3, 2, 1, 2, 1]))

6


In [ ]:
"""
More sophisticated version of the above approach using 2 pointers
No need for 2 extra arrays hence lesser space
Calculate local maxima and minima for each index using 2 pointers
"""
def trappedWater(num):
    if num == "":
        return 0
    i=0
    j=len(num)-1
    maxLeft=maxRight=0
    totalWater=0
    while i<j:
        maxLeft=max(maxLeft,num[i])
        maxRight=max(maxRight,num[j])
        if maxLeft < maxRight:
            totalWater += maxLeft-num[i]
            i+=1
        else:
            totalWater += maxRight-num[j]
            j-=1
    return totalWater

print("Total water trapped = " + str(trappedWater([0, 1, 0, 2, 1, 0, 1, 3, 2, 1, 2, 1])))

## Delete consecutive same words in sequence

Given a sequence of n strings, the task is to check if any two similar words come together then they destroy each other then print the number of words left in the sequence after this pairwise destruction.

Examples:

Input : ab aa aa bcd ab

Output : 3

As aa, aa destroys each other so, ab bcd ab is the
new sequence.

Input :  tom jerry jerry tom

Output : 0

As first both jerry will destroy each other.
Then sequence will be tom, tom they will also destroy
each other. So, the final sequence doesn't contain any
word.

In [13]:
def deleteConsecutive(strs):
    i=0
    stack=[]
    while i < len(strs):
        if stack and stack[-1] == strs[i]:
            stack.pop()
        else:
            stack.append(strs[i])
        i+=1
    return len(stack)

print(deleteConsecutive("tom jerry jerry tom".split()))

0


## Minimum remove to make valid parenthesis

Given a string s of '(' , ')' and lowercase English characters. 

Your task is to remove the minimum number of parentheses ( '(' or ')', in any positions ) so that the resulting parentheses string is valid and return any valid string.

Formally, a parentheses string is valid if and only if:

It is the empty string, contains only lowercase characters, or
It can be written as AB (A concatenated with B), where A and B are valid strings, or
It can be written as (A), where A is a valid string.
 

Example 1:


Input: s = "lee(t(c)o)de)"

Output: "lee(t(c)o)de"

Explanation: "lee(t(co)de)" , "lee(t(c)ode)" would also be accepted.


Example 2:

Input: s = "a)b(c)d"

Output: "ab(c)d"


Example 3:

Input: s = "))(("

Output: ""

Explanation: An empty string is also valid.


Example 4:

Input: s = "(a(b(c)d)"

Output: "a(b(c)d)"

In [12]:
def trim(expr, st):
    for k,v in enumerate(expr):
        if k in st:
            expr.pop(k)
    return ''.join(expr)

def removeParenthesis(expr):
    st=[]
    for k,v in enumerate(expr):
        if v == '(':
            st.append(k)
        elif v == ')':
            if st and expr[st[-1]]=='(':
                st.pop()
            else:
                st.append(k)
    return trim(expr,st)

print(removeParenthesis(list("(a(b(c)d)")))

a(b(c)d)


## Redundant Braces

Given a string A denoting an expression. It contains the following operators ’+’, ‘-‘, ‘*’, ‘/’.

Chech whether A has redundant braces or not.

Return 1 if A has redundant braces, else return 0.

Note: A will be always a valid expression.



Input Format

The only argument given is string A.

Output Format

Return 1 if string has redundant braces, else return 0.

For Example

Input 1:
    A = "((a + b))"

Output 1:
    1

Explanation 1:
        ((a + b)) has redundant braces so answer will be 1.

Input 2:
    A = "(a + (a + b))"

Output 2:
    0

Explanation 2:
        (a + (a + b)) doesn't have have any redundant braces so answer will be 0.

In [11]:
def checkRedundant(expr):
    st=[]
    for k,v in enumerate(expr):
        if v == ')':
            flag=False
            while st and st[-1] !='(':
                c = st.pop()
                if c in ['+','-','*','/']:
                    flag=True
            if not flag:
                return "There are redundant brackets"
            st.pop()
        st.append(v)
    return "There are no redundant brackets"

print(checkRedundant(list("(a+b*(c-d))")))

There are no redundant brackets


## Given an encoded string, return its decoded string.

The encoding rule is: k[encoded_string], where the encoded_string inside the square brackets is being repeated exactly k times. Note that k is guaranteed to be a positive integer.

You may assume that the input string is always valid; No extra white spaces, square brackets are well-formed, etc.

Furthermore, you may assume that the original data does not contain any digits and that digits are only for those repeat numbers, k. For example, there won't be input like 3a or 2[4].

 

Example 1:

Input: s = "3\[a\]2\[bc\]"

Output: "aaabcbc"


Example 2:

Input: s = "3\[a2\[c\]\]"

Output: "accaccacc"


Example 3:

Input: s = "2\[abc\]3\[cd\]ef"

Output: "abcabccdcdcdef"


Example 4:

Input: s = "abc3\[cd\]xyz"

Output: "abccdcdcdxyz"

In [9]:
"""
This solution is smart because it prevents the need for pushing each character into stack and reversing the stack to
reconstruct the string.
"""
def decodeString(expr):
    stack=[]
    out=''
    multiplier=0
    for c in expr: #Dont use enumerate if you dont need index k
        if c == '[': #Use elif instead of continue for each if block
            stack.append(out)
            stack.append(multiplier)#Append it now since u have the number already
            #Do this here and not when you see the closing bracket
            multiplier = 0
            out=''
        elif c == ']':
            num=stack.pop()
            prev=stack.pop()
            out = prev + num * out #And thus you prevent reversing the string
        elif c.isdigit(): #better than isnumeric since it is only ascii
            multiplier = multiplier*10 + int(c)
        else:
            out += c

    return out

print(decodeString(list("3[a2[c]]")))

accaccacc


## Simplify Path

Given a string path, which is an absolute path (starting with a slash '/') to a file or directory in a Unix-style file system, convert it to the simplified canonical path.

In a Unix-style file system, a period '.' refers to the current directory, a double period '..' refers to the directory up a level, and any multiple consecutive slashes (i.e. '//') are treated as a single slash '/'. For this problem, any other format of periods such as '...' are treated as file/directory names.

The canonical path should have the following format:

The path starts with a single slash '/'.
Any two directories are separated by a single slash '/'.
The path does not end with a trailing '/'.
The path only contains the directories on the path from the root directory to the target file or directory (i.e., no period '.' or double period '..')
Return the simplified canonical path.

 

Example 1:

Input: path = "/home/"

Output: "/home"

Explanation: Note that there is no trailing slash after the last directory name.


Example 2:

Input: path = "/../"

Output: "/"

Explanation: Going one level up from the root directory is a no-op, as the root level is the highest level you can go.


Example 3:

Input: path = "/home//foo/"

Output: "/home/foo"

Explanation: In the canonical path, multiple consecutive slashes are replaced by a single one.


Example 4:

Input: path = "/a/./b/../../c/"

Output: "/c"

In [10]:
"""
Simpler variation of decodeString
"""
def simplifyPath(expr):
    stack=[]
    for folder in expr.split('/'): #Had to take this approach instead to split by folders
        if stack and folder == '..':
            stack.pop()
        elif folder not in ['.', '..', '']: #Remember to add empty string too
            stack.append('/'+folder)
    out=''
    while stack:
        out = stack.pop() + out#append in reverse
    return out

print(simplifyPath("/a/./b/../c/"))

/c


## Basic Calculator

Given a string s representing an expression, implement a basic calculator to evaluate it.

Note: You are not allowed to use any built-in function which evaluates strings as mathematical expressions, such as eval().


Example 1:

Input: s = "1 + 1"

Output: 2

Example 2:

Input: s = " 2-1 + 2 "

Output: 3

Example 3:

Input: s = "(1+(4+5+2)-3)+(6+8)"

Output: 23

https://www.youtube.com/watch?v=081AqOuasw0
 

In [14]:
"""
Similar to decodeString but with more tricks
Algorithm:
1. Push result and sign every time you see open paren and set result=0 and sign=1
2. At every +,-and ) perform computation for result and adjust number=0 and sign=1
3. At closing paren also pop last 2 (result and sign) off stack and add to result
"""

def basicCalculator(expr):
    stack=[]
    number=0
    sign=1
    result=0
    for c in expr:
        if c == '(':
            stack.append(result)
            stack.append(sign)
            result = 0 #Set result = 0 only on open paren
            sign= 1
        elif c == ')':
            result += sign * number
            number=0 #Everywhere else set num = 0
            result *= stack.pop()
            result += stack.pop()
        elif c.isdigit():
            number = number*10 + int(c)
        elif c == '+':
            result += number*sign
            number=0
            sign = 1
        elif c == '-':
            result += number*sign
            number=0
            sign = -1 #You need this because sign is not left or right associative

    result+= number*sign
    return result

print(basicCalculator(list("(1+(4+5+2)-3)+(6+8)")))

23


## Max diff between nearest left and right smaller elements

Given an array of integers, the task is to find the maximum absolute difference between the nearest left and the right smaller element of every element in the array. 
Note: If there is no smaller element on right side or left side of any element then we take zero as the smaller element. For example for the leftmost element, the nearest smaller element on the left side is considered as 0. Similarly, for rightmost elements, the smaller element on the right side is considered as 0.
Examples: 
 

Input : arr[] = {2, 1, 8}

Output : 1

Left smaller  LS[] {0, 0, 1}

Right smaller RS[] {1, 0, 0}

Maximum Diff of abs(LS\[i\] - RS\[i\]) = 1 


Input  : arr[] = {2, 4, 8, 7, 7, 9, 3}

Output : 4

Left smaller   LS[] = {0, 2, 4, 4, 4, 7, 2}

Right smaller  RS[] = {0, 3, 7, 3, 3, 3, 0}

Maximum Diff of abs(LS\[i\] - RS\[i\]) = 7 - 3 = 4 


Input : arr[] = {5, 1, 9, 2, 5, 1, 7}

Output : 1

### Algorithm:

Very similar to Next Greater Element.  Find the result arrays for next smaller to right and to left (reverse input array). Then iterate through either array and return maxDiff


In [2]:
def nextSmallerElement(arr):
    st=[]
    st.append(0)
    res=[None]*len(arr)
    """
    Use indices instead in the below lines and return result array instead of printing
    Similar logic to NGE
    """
    for i in range(1,len(arr)):
        while st and arr[i] < arr[st[-1]]:
            res[st.pop()] = arr[i]
        st.append(i)
    while st:
        res[st.pop()] = -1
    return res

def maxDiffBetweenSmaller(arr):
    rightSmaller=nextSmallerElement(arr)
    leftSmaller=nextSmallerElement(arr[::-1]) #Pass in the reverse of Array
    maxDiff =-1
    for i in range(len(arr)):
        diff=abs(rightSmaller[i]-leftSmaller[i])
        maxDiff = max(maxDiff, diff)
    return maxDiff

print(maxDiffBetweenSmaller([4, 5, 2, 25]))

5
